In [1]:
import numpy as np
import time
import sys
import os
import tqdm
import operator

import torch
import torch.nn as nn
import torch.nn.functional as F
from queue import PriorityQueue

from dataset.dataset import ImageFeatureDataset
from torch.utils.data import DataLoader
from transformer_ethan import *
from catr.configuration import Config

In [2]:
words = np.load("glove_embed.npy")
with open('word2ind.json') as json_file: 
    word2ind = json.load(json_file) 
with open('ind2word.json') as json_file: 
    ind2word = json.load(json_file) 

config = Config()
config.device = 'cuda'
config.feature_dim = 1024
config.pad_token_id = word2ind["<S>"]
config.hidden_dim = 300
config.nheads = 10
config.batch_size = 64
config.encoder_type = 1
config.vocab_size = words.shape[0]
config.dir = '../mimic_features_double'
config.__dict__["pre_embed"] = torch.from_numpy(words).to(config.device)

In [3]:
model, criterion = main(config) 
model = model.float()
device = torch.device(config.device)
model.to(device)

Initializing Device: cuda
Number of params: 33370520


Xray_Captioner(
  (input_proj): Conv2d(1024, 300, kernel_size=(1, 1), stride=(1, 1))
  (position_embedding): PositionEmbeddingSine()
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): Linear(in_features=300, out_features=300, bias=True)
          )
          (linear1): Linear(in_features=300, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=300, bias=True)
          (norm1): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
        (1): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): Linear(in_features=300, out_features=3

In [12]:
dataset_val = ImageFeatureDataset(config, mode='val')
sampler_val = torch.utils.data.SequentialSampler(dataset_val)
data_loader_val = DataLoader(dataset_val, 1,
                             sampler=sampler_val, drop_last=False, num_workers=config.num_workers)

In [5]:
# Load from checkpoint
CHECKPOINT_NAME = "checkpoint_imagenet_10_tf.pth"

checkpoint = torch.load(CHECKPOINT_NAME, map_location=device)
model.load_state_dict(checkpoint['model'])

print("Loading Checkpoint...")

Loading Checkpoint...


In [6]:
import nltk

def get_bleu_score(truths, predicteds, weights_to_be_used=[0.25, 0.25, 0.25, 0.25]):
    scores = []
    for index in range(len(truths)):
        truth = truths[index]
        predicted = predicteds[index]
        try:
            score = nltk.translate.bleu_score.sentence_bleu([truth], predicted, weights=weights_to_be_used)
        except:
            score = 0
        scores.append(score)
    return sum(scores)/len(scores)

In [7]:
# Edward: note this makes a new caption as (<S>, 0, ..., 0) shouldn't we want as (<S>, <S>, ..., <S>)?
def create_caption_and_mask(start_token, max_length):
    caption_template = torch.zeros((1, max_length), dtype=torch.long)
    mask_template = torch.ones((1, max_length), dtype=torch.bool)

    caption_template[:, 0] = start_token
    mask_template[:, 0] = False

    return caption_template, mask_template

def make_report(captions):
    all_reports = []
    for report in captions:
        if (report == word2ind["</s>"]).any():
            end_index = (report == word2ind["</s>"]).nonzero()[0][0]
            report = report[:end_index+1]
        one_report = list(map(lambda x: ind2word[str(x)], report))
        all_reports.append(one_report)
    return all_reports

def reports_to_sentence(reports):
    return [' '.join(r) for r in make_report(reports)]

def evaluate(images):
    all_captions = []
    model.eval()
    for i in range(len(images)):
        image = images[i:i+1]
        caption, cap_mask = create_caption_and_mask(
            config.pad_token_id, config.max_position_embeddings)
#         caption.to("cuda")
        for i in range(config.max_position_embeddings - 1):
            with torch.no_grad():
                predictions = model(image, caption, cap_mask).to(config.device)
            predictions = predictions[:, i, :]
            predicted_id = torch.argmax(predictions, axis=-1)


            caption[:, i+1] = predicted_id[0]
            cap_mask[:, i+1] = False
            
            if predicted_id[0] == word2ind["</s>"]:
                break

        all_captions.append(caption.numpy())
#     return make_report(all_captions)
    return all_captions

def bleu(truth_in, generated_in):
    truth = truth_in.replace("<S>", "").replace("<s>", "").replace("</s>", "").replace(".", "").replace(",", "").replace("  ", " ").split(" ")
    generated = generated_in.replace("<S>", "").replace("<s>", "").replace("</s>", "").replace(".", "").replace(",", "").replace("  ", " ").split(" ")
    truth = [y for y in truth if y != ''] 
    generated = [y for y in generated if y != ''] 
    bs4 = nltk.translate.bleu_score.sentence_bleu([truth], generated, weights=[0.25, 0.25, 0.25, 0.25])
    bs3 = nltk.translate.bleu_score.sentence_bleu([truth], generated, weights=[1./3., 1./3., 1./3.])
    bs2 = nltk.translate.bleu_score.sentence_bleu([truth], generated, weights=[0.5, 0.5])
    bs1 = nltk.translate.bleu_score.sentence_bleu([truth], generated, weights=[1.])
    return bs1, bs2, bs3, bs4

In [8]:
import math

class BeamSearchNode(object):
    def __init__(self, hiddenstate, previousNode, wordId, logProb, length):
        '''
        :param hiddenstate:
        :param previousNode:
        :param wordId:
        :param logProb:
        :param length:
        '''
        self.h = hiddenstate
        self.prevNode = previousNode
        self.wordid = wordId
        self.logp = logProb
        self.leng = length

    def eval(self, alpha=1.0):
        return self.logp / float(self.leng - 1 + 1e-6)# * math.log(self.leng)
    
# def beam_decode(model, config, target_tensor, decoder_hiddens, encoder_outputs=None):
def beam_decode(model, image, beam_width=5, topk=1):
    '''
    :param target_tensor: target indexes tensor of shape [B, T] where B is the batch size and T is the maximum length of the output sentence
    :param decoder_hidden: input tensor of shape [1, B, H] for start of the decoding
    :param encoder_outputs: if you are using attention mechanism you can pass encoder outputs, [T, B, H] where T is the maximum length of input sentence
    :return: decoded_batch
    '''
    
    model.eval()

#     beam_width = 5
#     topk = 5  # how many sentence do you want to generate
    decoded_batch = []
    
    SOS_token = word2ind["<S>"]
    EOS_token = word2ind["</s>"]
    
    caption, cap_mask = create_caption_and_mask(
#             config.pad_token_id, config.max_position_embeddings)
            SOS_token, config.max_position_embeddings)
    caption[:,1:] = EOS_token
    
    # Number of sentence to generate
    endnodes = []
    number_required = min((topk + 1), topk - len(endnodes))
#     print(number_required, "required.")
    
    # starting node -  hidden vector, previous node, word id, logp, length
    node = BeamSearchNode(cap_mask, None, caption, 0, 1)
    nodes = PriorityQueue()

    # start the queue
    nodes.put((-node.eval(), node))
    qsize = 1
    
    # start beam search
    while True:
        # give up when decoding takes too long
        if qsize > 10000: 

            print("taking too long")
            break

        # fetch the best node
        score, n = nodes.get()
        decoder_input = n.wordid
        decoder_mask = n.h
#         decoder_hidden = n.h

        if n.leng == config.max_position_embeddings or n.wordid[0,n.leng-1].item() == EOS_token:
            endnodes.append((score, n))
            # if we reached maximum # of sentences required
            if len(endnodes) >= number_required:
                break
            else:
                continue

#         if n.wordid.item() == EOS_token and n.prevNode != None:
#             endnodes.append((score, n))
#             # if we reached maximum # of sentences required
#             if len(endnodes) >= number_required:
#                 break
#             else:
#                 continue

        # decode for one step using decoder
#         decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_output)

        
        with torch.no_grad():
            predictions = model(image, decoder_input, decoder_mask).to(config.device)
        predictions = predictions[0, n.leng-1, :] # shape: [22058]
#         predicted_id = torch.argmax(predictions, axis=-1)

        # PUT HERE REAL BEAM SEARCH OF TOP
        log_prob, indexes = torch.topk(predictions, beam_width)
        nextnodes = []

        for new_k in range(beam_width):
            predicted_id = indexes[new_k]
            log_p = log_prob[new_k].item()
            
            new_caption = decoder_input.detach().clone()
            new_mask = decoder_mask.detach().clone()
            
            new_caption[:, n.leng] = predicted_id
            new_mask[:, n.leng] = False

            node = BeamSearchNode(new_mask, n, new_caption, n.logp + log_p, n.leng + 1)
            score = -node.eval()
            nextnodes.append((score, node))

        # put them into queue
        for i in range(len(nextnodes)):
            score, nn = nextnodes[i]
            nodes.put((score, nn))
            # increase qsize
        qsize += len(nextnodes) - 1

    # choose nbest paths, back trace them
    if len(endnodes) < topk:
        for _ in range(topk-len(endnodes)):
            endnodes.append(nodes.get())

    print("Got ", len(endnodes), " end nodes")
    utterances = []
    for score, n in sorted(endnodes, key=operator.itemgetter(0)):
#         utterance = []
#         utterance.append(n.wordid)
#         # back trace
#         while n.prevNode != None:
#             n = n.prevNode
#             utterance.append(n.wordid)

#         utterance = utterance[::-1]
        utterances.append(n.wordid)

    return utterances
        
        
    # decoding goes sentence by sentence
    for idx in range(target_tensor.size(0)):
        if isinstance(decoder_hiddens, tuple):  # LSTM case
            decoder_hidden = (decoder_hiddens[0][:,idx, :].unsqueeze(0),decoder_hiddens[1][:,idx, :].unsqueeze(0))
        else:
            decoder_hidden = decoder_hiddens[:, idx, :].unsqueeze(0)
#         encoder_output = encoder_outputs[:,idx, :].unsqueeze(1)

        # Start with the start of the sentence token
        decoder_input = torch.LongTensor([[SOS_token]], device=device)

        # Number of sentence to generate
        endnodes = []
        number_required = min((topk + 1), topk - len(endnodes))

        # starting node -  hidden vector, previous node, word id, logp, length
        node = BeamSearchNode(decoder_hidden, None, decoder_input, 0, 1)
        nodes = PriorityQueue()

        # start the queue
        nodes.put((-node.eval(), node))
        qsize = 1

        # start beam search
        while True:
            # give up when decoding takes too long
            if qsize > 2000:
                print("taking too long")
                break

            # fetch the best node
            score, n = nodes.get()
            decoder_input = n.wordid
            decoder_hidden = n.h

            if n.wordid.item() == EOS_token and n.prevNode != None:
                endnodes.append((score, n))
                # if we reached maximum # of sentences required
                if len(endnodes) >= number_required:
                    print("Got all required")
                    break
                else:
                    continue

            # decode for one step using decoder
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_output)

            # PUT HERE REAL BEAM SEARCH OF TOP
            log_prob, indexes = torch.topk(decoder_output, beam_width)
            nextnodes = []

            for new_k in range(beam_width):
                decoded_t = indexes[0][new_k].view(1, -1)
                log_p = log_prob[0][new_k].item()

                node = BeamSearchNode(decoder_hidden, n, decoded_t, n.logp + log_p, n.leng + 1)
                score = -node.eval()
                nextnodes.append((score, node))

            # put them into queue
            for i in range(len(nextnodes)):
                score, nn = nextnodes[i]
                nodes.put((score, nn))
                # increase qsize
            qsize += len(nextnodes) - 1

        # choose nbest paths, back trace them
        if len(endnodes) == 0:
            endnodes = [nodes.get() for _ in range(topk)]

        utterances = []
        for score, n in sorted(endnodes, key=operator.itemgetter(0)):
            utterance = []
            utterance.append(n.wordid)
            # back trace
            while n.prevNode != None:
                n = n.prevNode
                utterance.append(n.wordid)

            utterance = utterance[::-1]
            utterances.append(utterance)

        decoded_batch.append(utterances)

    return decoded_batch

In [25]:
# ETHAN_NOTE = "<S> The heart size is normal . <s> The hilar and mediastinal contours are normal . <s> No focal consolidations concerning for pneumonia are identified . <s> There is no pleural effusion or pneumothorax . <s> The visualized osseous structures are unremarkable . <s> </s>"
ETHAN_NOTE = "<S> The heart size is normal . <s> The hilar and mediastinal contours are normal . <s> No focal consolidations . There is no pleural effusion or pneumothorax . <s> </s>"

def do_beam_iter(image, note, note_mask):
    
    beam_1 = []
    beam_5 = []
    beam_10 = []
    
    report = evaluate(image)
    report_np = np.asarray(report).squeeze(1)
    
    truth = reports_to_sentence(np.asarray(note[:,:]))[0]
    print('[GT]', truth)
    print()
    generated = reports_to_sentence(report_np)[0]
    print('[Beam 1 Pred]', generated)
    bs1, bs2, bs3, bs4 = bleu(truth, generated)
    print("[Beam 1] Bleu score: {0:.4f} {1:.4f} {2:.4f} {3:.4f}".format(bs1, bs2, bs3, bs4))
    
    beam_1 = [bs1, bs2, bs3, bs4]
    
    print()
    decoded_sentences = beam_decode(model, image, beam_width=5, topk=1)
    gen = reports_to_sentence(decoded_sentences[0].numpy())[0]
    print("[Beam 5 Pred] ",gen)
    bs1, bs2, bs3, bs4 = bleu(truth, gen)
    beam_5 = [bs1, bs2, bs3, bs4]
    print("[Beam 5] Bleu score: {0:.4f} {1:.4f} {2:.4f} {3:.4f}".format(bs1, bs2, bs3, bs4))
        
#     print()
#     decoded_sentences = beam_decode(model, image, beam_width=10, topk=1)
#     gen = reports_to_sentence(decoded_sentences[0].numpy())[0]
#     print("[Beam 10 Pred] ",gen)
#     bs1, bs2, bs3, bs4 = bleu(truth, gen)
#     beam_10 = [bs1, bs2, bs3, bs4]
#     print("[Beam 10] Bleu score: {0:.4f} {1:.4f} {2:.4f} {3:.4f}".format(bs1, bs2, bs3, bs4))
#     print()
    
    bs1, bs2, bs3, bs4 = bleu(truth, ETHAN_NOTE)
    beam_ethan = [bs1, bs2, bs3, bs4]
    print("[Beam ethan] Bleu score: {0:.4f} {1:.4f} {2:.4f} {3:.4f}".format(bs1, bs2, bs3, bs4))
    
        
    print('---------------------------------------------------------------')
    print()
    
    return beam_1, beam_5, beam_10, beam_ethan

In [22]:
bleu_scores_all = []
for image, note, note_mask in tqdm.tqdm(data_loader_val):
    
    image = image.cuda()
    
    report = evaluate(image)
    report_np = np.asarray(report).squeeze(1)
    
    truth = reports_to_sentence(np.asarray(note[:,:]))[0]
    generated = reports_to_sentence(report_np)[0]
    bs1, bs2, bs3, bs4 = bleu(truth, generated)
    bleu_scores_all.append([bs1, bs2, bs3, bs4])

  0%|          | 0/1196 [00:00<?, ?it/s]/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
  1%|          | 14/1196 [00:19<21:23,  1.09s/it]/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BL

In [23]:
print("Blue Scores: ", np.asarray(bleu_scores_all).mean(0))

Blue Scores:  [0.24713465 0.15262508 0.09833493 0.06841115]


In [20]:
len(data_loader_val)

1196

In [26]:

beam_1_list = []
beam_5_list = []
beam_10_list = []
beam_ethan_list = []

iters = 0
for image, note, note_mask in data_loader_val:
    
    image = image.cuda()
    
    
    beam_1, beam_5, beam_10, beam_ethan = do_beam_iter(image, note, note_mask)
    beam_1_list.append(beam_1)
    beam_5_list.append(beam_5)
#     beam_10_list.append(beam_10)
    beam_ethan_list.append(beam_ethan)

    iters += 1
    if iters >= 128:
        break
    

[GT] <S> No evidence of consolidation to suggest pneumonia is seen . <s> There is some retrocardiac atelectasis . <s> A small left pleural effusion may be present . <s> No pneumothorax is seen . <s> No pulmonary edema . <s> A right granuloma is unchanged . <s> The heart is mildly enlarged , unchanged . <s> There is tortuosity of the aorta . <s> </s>

[Beam 1 Pred] <S> Frontal and lateral views of the chest were obtained . <s> There is persistent elevation of the left hemidiaphragm . <s> There is blunting of the left costophrenic angle which may be due to a trace pleural effusion . <s> There is also a trace right pleural effusion . <s> The cardiac silhouette is top normal to mildly enlarged . <s> The aorta is calcified and tortuous . <s> There is mild pulmonary vascular congestion . <s> </s>
[Beam 1] Bleu score: 0.3651 0.1880 0.0000 0.0000



/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Got  1  end nodes
[Beam 5 Pred]  <S> Frontal and lateral views of the chest . <s> Heart size and cardiomediastinal contours are normal . <s> Lung volumes are low . <s> There is bibasilar atelectasis . <s> No pleural effusion or pneumothorax . <s> </s>
[Beam 5] Bleu score: 0.1649 0.0970 0.0000 0.0000
[Beam ethan] Bleu score: 0.1374 0.0812 0.0000 0.0000
---------------------------------------------------------------

[GT] <S> There are moderate bilateral pleural effusions with overlying atelectasis , underlying consolidation not excluded . <s> Mild prominence of the interstitial markings suggests mild pulmonary edema . <s> The cardiac silhouette is mildly enlarged . <s> The mediastinal contours are unremarkable . <s> There is no evidence of pneumothorax . <s> </s>

[Beam 1 Pred] <S> AP and lateral views of the chest . <s> There are bilateral pleural effusions , moderate on the left and small on the right . <s> There is pulmonary vascular congestion . <s> Cardiac silhouette is enlarged bu

[GT] <S> No previous images . <s> There are relatively low lung volumes which may accentuate the prominent transverse diameter of the heart . <s> No evidence of vascular congestion or pleural effusion . <s> No convincing evidence of acute pneumonia , though there are some <unk> changes at the left base . <s> </s>

[Beam 1 Pred] <S> As compared to the previous radiograph , there is no relevant change . <s> The lung volumes are low . <s> There is no evidence of pneumonia or other acute lung disease . <s> Borderline size of the cardiac silhouette . <s> No pleural effusions . <s> No pulmonary edema . <s> </s>
[Beam 1] Bleu score: 0.4165 0.1723 0.0000 0.0000

Got  1  end nodes
[Beam 5 Pred]  <S> As compared to the previous radiograph , there is no relevant change . <s> Low lung volumes . <s> Borderline size of the cardiac silhouette without pulmonary edema . <s> No pleural effusions . <s> </s>
[Beam 5] Bleu score: 0.1800 0.0865 0.0000 0.0000
[Beam ethan] Bleu score: 0.1225 0.0474 0.0000 0.0

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/nltk/translate/bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


...

[GT] <S> Interval extubation . <s> <unk> contours are within normal limits and without change . <s> Improved aeration at the lung bases with residual subsegmental atelectasis at the left lung base . <s> </s>

[Beam 1 Pred] <S> As compared to the previous radiograph , there is no relevant change . <s> No evidence of pneumonia . <s> No pleural effusions . <s> No pulmonary edema . <s> Normal size of the cardiac silhouette . <s> </s>
[Beam 1] Bleu score: 0.1111 0.0000 0.0000 0.0000

Got  1  end nodes
[Beam 5 Pred]  <S> As compared to the previous radiograph , there is no relevant change . <s> Borderline size of the cardiac silhouette without pulmonary edema . <s> No pleural effusions . <s> </s>
[Beam 5] Bleu score: 0.1526 0.0000 0.0000 0.0000
[Beam ethan] Bleu score: 0.1516 0.0776 0.0000 0.0000
---------------------------------------------------------------

[GT] <S> The lungs are clear without focal consolidation . <s> No pleural effusion or pneumothorax is seen . <s> The heart is <unk> i

[GT] <S> The lungs show minimal bilateral dependent atelectasis . <s> The lungs are otherwise clear . <s> <unk> silhouette and hilar contours are unremarkable . <s> No pleural effusion or pneumothorax . <s> No evidence of free air . <s> </s>

[Beam 1 Pred] <S> As compared to the previous radiograph , there is no relevant change . <s> The lung volumes remain low . <s> Borderline size of the cardiac silhouette without pulmonary edema . <s> No pleural effusions . <s> No pneumonia . <s> No pneumothorax . <s> </s>
[Beam 1] Bleu score: 0.2187 0.0840 0.0000 0.0000

Got  1  end nodes
[Beam 5 Pred]  <S> As compared to the previous radiograph , there is no relevant change . <s> Borderline size of the cardiac silhouette without pulmonary edema . <s> No pleural effusions . <s> No pneumonia . <s> </s>
[Beam 5] Bleu score: 0.1704 0.0778 0.0000 0.0000
[Beam ethan] Bleu score: 0.3637 0.2245 0.1542 0.1089
---------------------------------------------------------------

[GT] <S> There is no focal consol

[GT] <S> Compared to the prior study there is no significant interval change . <s> </s>

[Beam 1 Pred] <S> As compared to the previous radiograph , there is no relevant change . <s> The lung volumes have slightly decreased . <s> Moderate cardiomegaly with bilateral pleural effusions and areas of atelectasis at the lung bases . <s> No evidence of pneumonia . <s> No pneumothorax . <s> </s>
[Beam 1] Bleu score: 0.1622 0.1162 0.0728 0.0000

Got  1  end nodes
[Beam 5 Pred]  <S> As compared to the previous radiograph , there is no relevant change . <s> Moderate pulmonary edema persists . <s> Moderate cardiomegaly . <s> No larger pleural effusions . <s> </s>
[Beam 5] Bleu score: 0.2857 0.2070 0.1311 0.0000
[Beam ethan] Bleu score: 0.0909 0.0658 0.0000 0.0000
---------------------------------------------------------------

[GT] <S> Lungs are clear . <s> The cardiomediastinal silhouette is within normal limits . <s> No acute osseous abnormalities . <s> </s>

[Beam 1 Pred] <S> The lungs are clea

Got  1  end nodes
[Beam 5 Pred]  <S> AP upright and lateral views of the chest were reviewed . <s> Lung volumes are low . <s> There is no pleural effusion or pneumothorax . <s> The cardiomediastinal and hilar contours are normal . <s> Pulmonary vascularity is normal . <s> </s>
[Beam 5] Bleu score: 0.1826 0.1450 0.1221 0.1005
[Beam ethan] Bleu score: 0.0730 0.0464 0.0298 0.0000
---------------------------------------------------------------

[GT] <S> ET tube is seen with tip approximately 1.8 cm from the carina . <s> Enteric tube seen passing below the inferior field of view . <s> Lower lung volumes are noted on the current exam with bilateral parenchymal opacities which could be due to edema or infection . <s> Prominence of the right hilum is again noted . <s> Moderate cardiomegaly and appears to have progressed since prior could potentially be in part due to changes in positioning . <s> No acute osseous abnormalities . <s> Surgical clips project over the left chest <unk> . <s> </s>

[

Got  1  end nodes
[Beam 5 Pred]  <S> As compared to the previous radiograph , there is no relevant change . <s> No evidence of pneumonia . <s> No pulmonary edema . <s> No pleural effusions . <s> Borderline size of the cardiac silhouette . <s> </s>
[Beam 5] Bleu score: 0.1902 0.0867 0.0000 0.0000
[Beam ethan] Bleu score: 0.1598 0.0896 0.0000 0.0000
---------------------------------------------------------------

[GT] <S> Right-sided pleural effusion has minimally decreased . <s> Right-sided adjacent atelectasis and fluid along the fissure have also decreased . <s> The left lung is clear . <s> The cardiomediastinal silhouette is unchanged . <s> Numerous calcified lesions in the right chest wall are stable . <s> </s>

[Beam 1 Pred] <S> PA and lateral views of the chest provided . <s> There is a small right pleural effusion with associated compressive atelectasis . <s> There is no focal consolidation concerning for pneumonia . <s> Heart size is normal . <s> Mediastinal and hilar contours a

In [ ]:
'''
Beam 1 Bleu score: 0.3220 0.2219 0.1090 0.0000
Beam 5 Bleu score: 0.0909 0.0615 0.0317 0.0000

Beam 1 Bleu score: 0.1315 0.0771 0.0517 0.0000
Beam 5 Bleu score: 0.0323 0.0162 0.0000 0.0000

Beam 1 Bleu score: 0.2559 0.1497 0.0876 0.0000
Beam 5 Bleu score: 0.0812 0.0501 0.0273 0.0000

Beam 1 Bleu score: 0.1667 0.0595 0.0000 0.0000
Beam 5 Bleu score: 0.1600 0.0808 0.0514 0.0000

Beam 1 Bleu score: 0.2652 0.1967 0.1603 0.1376
Beam 5 Bleu score: 0.1225 0.1083 0.0992 0.0908

Beam 1 Bleu score: 0.0869 0.0553 0.0363 0.0000
Beam 5 Bleu score: 0.1048 0.0595 0.0291 0.0000
'''


In [27]:
np.asarray(beam_1_list).mean(0)

array([0.23706058, 0.14838034, 0.09575199, 0.06454364])

In [28]:
np.asarray(beam_5_list).mean(0)

array([0.19125346, 0.12235446, 0.07711691, 0.05343355])

In [18]:
np.asarray(beam_10_list).mean(0)

array([0.18192014, 0.11734206, 0.07463041, 0.05390324])

In [29]:
np.asarray(beam_ethan_list).mean(0)

array([0.1682474 , 0.0963156 , 0.05805758, 0.03095153])

In [187]:
image, note, note_mask = next(iter(data_loader_val))

In [190]:
report = evaluate(image)
report_np = np.asarray(report).squeeze(1)

In [194]:
truth = reports_to_sentence(np.asarray(note[:,:]))[0]
truth

'<S> No evidence of consolidation to suggest pneumonia is seen . <s> There is some retrocardiac atelectasis . <s> A small left pleural effusion may be present . <s> No pneumothorax is seen . <s> No pulmonary edema . <s> A right granuloma is unchanged . <s> The heart is mildly enlarged , unchanged . <s> There is tortuosity of the aorta . <s> </s>'

In [195]:
generated = reports_to_sentence(report_np)[0]
generated

'<S> AP upright and lateral views of the chest provided . <s> Lung volumes are low . <s> There is mild left basal atelectasis . <s> No convincing evidence for pneumonia or edema . <s> No large effusion or pneumothorax . <s> The cardiomediastinal silhouette is stable . <s> Imaged osseous structures are intact . <s> No free air below the right hemidiaphragm is seen . <s> </s>'

In [196]:
bs1, bs2, bs3, bs4 = bleu(truth, generated)
print("Bleu score: {0:.4f} {1:.4f} {2:.4f} {3:.4f}".format(bs1, bs2, bs3, bs4))

Bleu score: 0.3800 0.1525 0.0000 0.0000


In [198]:
decoded_sentences = beam_decode(model, image)

5 required.
Got  5  end nodes


In [203]:
for i in range(len(decoded_sentences)):
    gen = reports_to_sentence(decoded_sentences[i].numpy())[0]
    print("[iter ",i,"] ",gen)
    bs1, bs2, bs3, bs4 = bleu(truth, gen)
    print("[BLEU] Bleu score: {0:.4f} {1:.4f} {2:.4f} {3:.4f}".format(bs1, bs2, bs3, bs4))

[iter  0 ]  <S> AP upright and lateral views of the chest were obtained . <s> Lung volumes are low . <s> No focal consolidation is seen . <s> No pleural effusion or pneumothorax . <s> The cardiac and mediastinal silhouettes are stable . <s> No overt pulmonary edema is seen . <s> </s>
[BLEU] Bleu score: 0.3391 0.2105 0.0998 0.0000
[iter  1 ]  <S> AP upright and lateral views of the chest were obtained . <s> Lung volumes are low . <s> No focal consolidation is seen . <s> No pleural effusion or pneumothorax . <s> The cardiac and mediastinal silhouettes are stable . <s> No overt pulmonary edema is seen . <s> No displaced fracture is identified . <s> </s>
[BLEU] Bleu score: 0.3680 0.2213 0.1036 0.0000
[iter  2 ]  <S> AP upright and lateral views of the chest were obtained . <s> Lung volumes are low . <s> No focal consolidation is seen . <s> No pleural effusion or pneumothorax . <s> The cardiac and mediastinal silhouettes are stable . <s> No overt pulmonary edema is seen . <s> No displaced f

[iter  0 ]  <S> AP upright and lateral views of the chest were obtained . <s> Lung volumes are low . <s> No focal consolidation is seen . <s> No pleural effusion or pneumothorax . <s> The cardiac and mediastinal silhouettes are stable . <s> No overt pulmonary edema is seen . <s> </s>
[BLEU] Bleu score: 0.3391 0.2105 0.0998 0.0000
[iter  1 ]  <S> AP upright and lateral views of the chest were obtained . <s> Lung volumes are low . <s> No focal consolidation is seen . <s> No pleural effusion or pneumothorax . <s> The cardiac and mediastinal silhouettes are stable . <s> No overt pulmonary edema is seen . <s> No displaced fracture is identified . <s> </s>
[BLEU] Bleu score: 0.3680 0.2213 0.1036 0.0000
[iter  2 ]  <S> AP upright and lateral views of the chest were obtained . <s> Lung volumes are low . <s> No focal consolidation is seen . <s> No pleural effusion or pneumothorax . <s> The cardiac and mediastinal silhouettes are stable . <s> No overt pulmonary edema is seen . <s> No displaced f

In [172]:
truth

'<S> No evidence of consolidation to suggest pneumonia is seen . <s> There is some retrocardiac atelectasis . <s> A small left pleural effusion may be present . <s> No pneumothorax is seen . <s> No pulmonary edema . <s> A right granuloma is unchanged . <s> The heart is mildly enlarged , unchanged . <s> There is tortuosity of the aorta . <s> </s>'